## RETO - CAMPAÑA DE MARKETING DE UN BANCO PORTUGUÉS

## EDA y limpieza de los dos sets de datos

- 1 Carga de librerías
- 2 EDA del primer conjunto de datos ('bank-additional.csv')
- 3 EDA del segundo conjunto de datos ('customer-details.xlsx')
- 4 Unión de los dos datasets
- 5 Limpieza de los datasets

## 1 Carga de librerías

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)

## 2 EDA del primer conjunto de datos ('bank-additional.csv')

In [2]:
bank = pd.read_csv('DataSource/bank-additional.csv',index_col = 0)
print(f"El número de filas que tenemos es {bank.shape[0]}, y el número de columnas es {bank.shape[1]}")


El número de filas que tenemos es 43000, y el número de columnas es 23


In [3]:
bank.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'date', 'latitude', 'longitude', 'id_'],
      dtype='object')

In [4]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43000 entries, 0 to 27570
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             37880 non-null  float64
 1   job             42655 non-null  object 
 2   marital         42915 non-null  object 
 3   education       41193 non-null  object 
 4   default         34019 non-null  float64
 5   housing         41974 non-null  float64
 6   loan            41974 non-null  float64
 7   contact         43000 non-null  object 
 8   duration        43000 non-null  int64  
 9   campaign        43000 non-null  int64  
 10  pdays           43000 non-null  int64  
 11  previous        43000 non-null  int64  
 12  poutcome        43000 non-null  object 
 13  emp.var.rate    43000 non-null  float64
 14  cons.price.idx  42529 non-null  object 
 15  cons.conf.idx   43000 non-null  object 
 16  euribor3m       33744 non-null  object 
 17  nr.employed     43000 non-null  obje

## Conclusiones que sacamos con el .info()

a) Columnas con incluyen nulos:
- age
- job
- marital
- education
- default
- housing
- loan
- cons.price.idx
- euribor3m
- date

b) Columnas con dtype que no son coherentes:
- cons.price.idx
- cons.conf.idx
- euribor3m
- nr.employed
- date

In [5]:
# Valores únicos y conteo de éstos de todas las columnas categóricas.
bank_cat = bank.select_dtypes(include = 'O')
lista_col = bank_cat.columns.tolist()
for col in lista_col:
    print(f'Para la columna {col.upper()}:\n')# Printeamos el nombre de la columna en mayúsculas.
    print(f'Los valóres únicos son: {bank[col].unique()}\n')# Mostramos los valores únicos de la columna
    print(f'El conteo de valores únicos es:\n{bank[col].value_counts()}') # Observamos el conteo de cada valor único
    print('-----------')

Para la columna JOB:

Los valóres únicos son: ['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' nan 'entrepreneur' 'student']

El conteo de valores únicos es:
job
admin.           10873
blue-collar       9654
technician        7026
services          4162
management        3050
retired           1790
entrepreneur      1522
self-employed     1489
housemaid         1123
unemployed        1063
student            903
Name: count, dtype: int64
-----------
Para la columna MARITAL:

Los valóres únicos son: ['MARRIED' 'SINGLE' 'DIVORCED' nan]

El conteo de valores únicos es:
marital
MARRIED     25999
SINGLE      12105
DIVORCED     4811
Name: count, dtype: int64
-----------
Para la columna EDUCATION:

Los valóres únicos son: ['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course' nan
 'university.degree' 'illiterate']

El conteo de valores únicos es:
education
university.degree      12722
high.school             9925
ba

# Conclusiones de los valores únicos

- **job**: El resultado principal es 'admin', pero no con una diferencia significativa del siguiente. Hay nulos
- **marital**: Sólo tenemos 4 valores (MARRIED, SINGLE, DIVORCED y nan), el primero de ellos destaca muy por encima del resto. Hay nulos
- **education**: El resultado principal es 'university degree', es destacable que 'illiterate' solo posee 18 registros. Hay nulos
- **default**: Sólo tenemos dos valores (No y Si), el segundo sólo aparece en 3 registros.
- **housing**: Tres valores (No, Sí y Unknown).
- **loan**: Dos valores únicos (No y Sí), el primero notablemente más presente que el segundo.
- **contact**: Dos valores (telephone y cellular). El segundo tiene casi el doble de registros que el primero.
- **poutcome**: Tres valores (NONEXISTENT, FAILURE, SUCCESS). Tenemos notablemente más datos del primero que del resto.
- **cons.price.idx**: Tiene nulos. Es una falsa columna categórica.
- **cons.conf.idx**: Es una falsa columna categórica.
- **euribor3m**: Tiene nulos. Es una falsa columna categórica.
- **nr.employed**: Tiene nulos. Es una falsa columna categórica.
- **y**: Dos valores (no, yes). Gran cantidad de noes frente a síes.
- **date**: Falsa columna categórica.
- **id_**: Cada valor sólo aparece una vez, es decir, no tenemos dos registros con el mismo identificador.

In [6]:
# Calcula el número de nulos (solo de las columnnas que tengan al menos uno)
bank_nulos = bank.isnull().sum()
df_nulos = pd.DataFrame(bank_nulos, columns = ["total_nulos"])
df_nulos = df_nulos[df_nulos["total_nulos"] > 0]
df_nulos

,total_nulos
age,5120
job,345
marital,85
education,1807
default,8981
housing,1026
loan,1026
cons.price.idx,471
euribor3m,9256
date,248


## 3 EDA del segundo conjunto de datos ('customer-details.xlsx')
- Concatena las tres hojas del conjunto de datos
- Carga del archivo, mirar dimensiones, columnas, datos estadísticos, nulos y duplicados

In [7]:
# Crearemos ahora 3 DF distintos, uno para cada año (y hoja) de los datos de los clientes.
customer_2012 = pd.read_excel("DataSource/customer-details.xlsx", sheet_name="2012", index_col = 0)
customer_2013 = pd.read_excel("DataSource/customer-details.xlsx", sheet_name="2013", index_col = 0)
customer_2014 = pd.read_excel("DataSource/customer-details.xlsx", sheet_name="2014", index_col = 0)

customer_concat = pd.concat([customer_2012, customer_2013, customer_2014], axis = 0, ignore_index = True)
customer_concat.head()

,Income,Kidhome,Teenhome,Dt_Customer,NumWebVisitsMonth,ID
0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079
1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097
2,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,121393,1,2,2012-12-21,29,9991fafb-4447-451a-8be2-b0df6098d13e
4,63164,1,2,2012-06-20,20,eca60b76-70b6-4077-80ba-bc52e8ebb0eb


In [8]:
print(f"El número de filas que tenemos en customer_concat es {customer_concat.shape[0]}, y el número de columnas es {customer_concat.shape[1]}")

El número de filas que tenemos en customer_concat es 43170, y el número de columnas es 6


In [9]:
customer_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43170 entries, 0 to 43169
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Income             43170 non-null  int64         
 1   Kidhome            43170 non-null  int64         
 2   Teenhome           43170 non-null  int64         
 3   Dt_Customer        43170 non-null  datetime64[ns]
 4   NumWebVisitsMonth  43170 non-null  int64         
 5   ID                 43170 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 2.0+ MB


In [10]:
customer_concat.isnull().sum()

Income               0
Kidhome              0
Teenhome             0
Dt_Customer          0
NumWebVisitsMonth    0
ID                   0
dtype: int64

In [11]:
customer_concat.duplicated().sum() 

0

In [12]:
# Vamos ahora a comparar las dimensiones del DataFrame de bank con el de customer_concat
print(f"El número de filas que tenemos en bank es {bank.shape[0]}, y el número de columnas es {bank.shape[1]}")
print(f"El número de filas que tenemos en customer_concat es {customer_concat.shape[0]}, y el número de columnas es {customer_concat.shape[1]}")

El número de filas que tenemos en bank es 43000, y el número de columnas es 23
El número de filas que tenemos en customer_concat es 43170, y el número de columnas es 6


# Conclusiones previas a unir los dos datasets
- Aquí podemos ver que el número de filas de uno y otro es distinto. ¿A qué puede ser debido?
- Recordemos que el primero son datos de clientes a lo que se ha contactado para la campaña, mientras que el segundo son datos de clientes del banco.
- En este caso es más conveniente unir los DF con un merge del tipo 'inner' ya que nos interesan los valores coincidentes de ambos DF. ¿Por qué columna unimos?
- Uniremos por la columna que nos da el número identificativo del cliente, es decir, 'id_' del DF bank e 'ID' del DF customer_concat

## 4 Unión de datos
- Hacer uso de merge para la unión de los dos conjuntos de datos

In [13]:
bank = bank.merge(customer_concat,how = 'inner', left_on = 'id_', right_on = "ID")
bank.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_,Income,Kidhome,Teenhome,Dt_Customer,NumWebVisitsMonth,ID
0,NaN,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857",5191,no,2-agosto-2019,41.495,-71.233,089b39d8-e4d0-461b-87d4-814d71e0e079,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079
1,57.0,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,14-septiembre-2016,34.601,-83.923,e9d37224-cb6f-4942-98d7-46672963d097,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097
2,37.0,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857",5191,no,15-febrero-2019,34.939,-94.847,3f9f49b5-e410-4948-bf6e-f9244f04918b,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,40.0,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,29-noviembre-2015,49.041,-70.308,9991fafb-4447-451a-8be2-b0df6098d13e,121393,1,2,2012-12-21,29,9991fafb-4447-451a-8be2-b0df6098d13e
4,56.0,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,29-enero-2017,38.033,-104.463,eca60b76-70b6-4077-80ba-bc52e8ebb0eb,63164,1,2,2012-06-20,20,eca60b76-70b6-4077-80ba-bc52e8ebb0eb


## 5 Limpieza de los datos.
- Elimina las columnas redundantes (drop)
- Homogeneiza los nombres de las columnas (replace)
- Haz que todos los datos estén en minúsculas. Para ello utiliza .str.lower()
- Crea nuevas columnas para el mes('contact_month') y el año('contact_year') de la columna 'date' (split)
- Convierte la columna 'date' a tipo datetime con todatetime para cambiar el tipo de dato de la columna (Recuerda usar el formato %d-%m-%Y)
- Reemplaza los . por espacios en la columna 'education'. Utiliza.str.replace()
- Convierte a float las columnas que deberían serlo (apply con función lambda)
- Mapear los valores 0 y 1 por No y Sí.
- Genera nulos en la columna 'pdays'

In [14]:
# 5.1 Elimina columnas redundantes
bank.drop("id_", axis = 1, inplace = True)

In [15]:
# 5.2 Homogeneiza los nombres de las columnas: mayusculas con minúsculas, sin espacios y cambiar '.' por '_'
nuevas_columnas = {columna: columna.lower().replace(".", "_") for columna in bank.columns}
bank.rename(columns = nuevas_columnas, inplace = True)
bank.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m',
       'nr_employed', 'y', 'date', 'latitude', 'longitude', 'income',
       'kidhome', 'teenhome', 'dt_customer', 'numwebvisitsmonth', 'id'],
      dtype='object')

In [16]:
# 5.3 Haz que todos los datos estén en minúsculas. Para ello utiliza .str.lower()
for col in bank.columns.to_list():
    try:
        bank[col] = bank[col].str.lower()
    except:
        pass
bank.head(2)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,date,latitude,longitude,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id
0,NaN,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,nonexistent,1.1,"93,994","-36,4","4,857",5191,no,2-agosto-2019,41.495,-71.233,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079
1,57.0,services,married,high.school,NaN,0.0,0.0,telephone,149,1,999,0,nonexistent,1.1,"93,994","-36,4",NaN,5191,no,14-septiembre-2016,34.601,-83.923,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097


In [17]:
# 5.4 Crea nuevas columnas para el mes('contact_month') y el año('contact_year') de la columna 'date'. 
fecha = bank['date'].str.split('-', expand = True)
bank['contact_month'] = fecha[1]
bank['contact_year'] = fecha[2]
bank.head(2)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,date,latitude,longitude,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,contact_month,contact_year
0,NaN,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,nonexistent,1.1,"93,994","-36,4","4,857",5191,no,2-agosto-2019,41.495,-71.233,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,agosto,2019
1,57.0,services,married,high.school,NaN,0.0,0.0,telephone,149,1,999,0,nonexistent,1.1,"93,994","-36,4",NaN,5191,no,14-septiembre-2016,34.601,-83.923,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,septiembre,2016


In [18]:
# 5.5 Convierte la columna 'date' a tipo datetime.
meses = {'enero':'01',
       'febrero':'02',
       'marzo':'03',
       'abril':'04',
       'mayo':'05',
       'junio':'06',
       'julio':'07',
       'agosto':'08',
       'septiembre':'09',
       'octubre':'10',
       'noviembre':'11',
       'diciembre':'12'}

bank.replace({'date':meses}, regex = True,inplace = True)
bank['date'] = pd.to_datetime(bank['date'], format = "%d-%m-%Y") 
bank.head(2)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,date,latitude,longitude,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,contact_month,contact_year
0,NaN,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,nonexistent,1.1,"93,994","-36,4","4,857",5191,no,2019-08-02,41.495,-71.233,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,agosto,2019
1,57.0,services,married,high.school,NaN,0.0,0.0,telephone,149,1,999,0,nonexistent,1.1,"93,994","-36,4",NaN,5191,no,2016-09-14,34.601,-83.923,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,septiembre,2016


In [19]:
bank['date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 43000 entries, 0 to 42999
Series name: date
Non-Null Count  Dtype         
--------------  -----         
42752 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 336.1 KB


In [20]:
# 5.6 Reemplaza los . por espacios en la columna 'education'. Utiliza.str.replace()
bank['education'] = bank['education'].str.replace('.',' ')
bank.sample()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,date,latitude,longitude,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,contact_month,contact_year
31548,45.0,management,married,university degree,0.0,1.0,0.0,cellular,239,1,999,0,nonexistent,-1.8,"92,893","-46,2","1,327","5099,1",no,2017-10-28,35.709,-112.722,162315,0,2,2014-11-19,19,8b943ef8-b205-477b-83e8-2aa28adc842d,octubre,2017


In [21]:
# 5.7 Convierte a float las columnas que deberían serlo
str_float = ['cons_price_idx','cons_conf_idx','euribor3m','nr_employed']
for col in str_float:
    bank[col] = bank[col].str.replace(',','.')
    bank[col] = bank[col].apply(lambda x: float(x))
bank.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,date,latitude,longitude,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,contact_month,contact_year
0,NaN,housemaid,married,basic 4y,0.0,0.0,0.0,telephone,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2019-08-02,41.495,-71.233,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,agosto,2019
1,57.0,services,married,high school,NaN,0.0,0.0,telephone,149,1,999,0,nonexistent,1.1,93.994,-36.4,NaN,5191.0,no,2016-09-14,34.601,-83.923,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,septiembre,2016
2,37.0,services,married,high school,0.0,1.0,0.0,telephone,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2019-02-15,34.939,-94.847,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b,febrero,2019
3,40.0,admin.,married,basic 6y,0.0,0.0,0.0,telephone,151,1,999,0,nonexistent,1.1,93.994,-36.4,NaN,5191.0,no,2015-11-29,49.041,-70.308,121393,1,2,2012-12-21,29,9991fafb-4447-451a-8be2-b0df6098d13e,noviembre,2015
4,56.0,services,married,high school,0.0,0.0,1.0,telephone,307,1,999,0,nonexistent,1.1,93.994,-36.4,NaN,5191.0,no,2017-01-29,38.033,-104.463,63164,1,2,2012-06-20,20,eca60b76-70b6-4077-80ba-bc52e8ebb0eb,enero,2017


In [22]:
# 5.8 Mapear los valores 0 y 1 por No y Sí.
diccionario_mapeo = {0: "No", 1: "Si"}
mapear = ['default','housing','loan']
for col in mapear:
    bank[col] = bank[col].map(diccionario_mapeo)

bank.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,date,latitude,longitude,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,contact_month,contact_year
0,NaN,housemaid,married,basic 4y,No,No,No,telephone,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2019-08-02,41.495,-71.233,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,agosto,2019
1,57.0,services,married,high school,NaN,No,No,telephone,149,1,999,0,nonexistent,1.1,93.994,-36.4,NaN,5191.0,no,2016-09-14,34.601,-83.923,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,septiembre,2016
2,37.0,services,married,high school,No,Si,No,telephone,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2019-02-15,34.939,-94.847,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b,febrero,2019
3,40.0,admin.,married,basic 6y,No,No,No,telephone,151,1,999,0,nonexistent,1.1,93.994,-36.4,NaN,5191.0,no,2015-11-29,49.041,-70.308,121393,1,2,2012-12-21,29,9991fafb-4447-451a-8be2-b0df6098d13e,noviembre,2015
4,56.0,services,married,high school,No,No,Si,telephone,307,1,999,0,nonexistent,1.1,93.994,-36.4,NaN,5191.0,no,2017-01-29,38.033,-104.463,63164,1,2,2012-06-20,20,eca60b76-70b6-4077-80ba-bc52e8ebb0eb,enero,2017


# 5.9 Genera nulos en la columna 'pdays'.
No tiene mucho sentido que hayan pasado 999 días desde el último contacto, probablemente es la forma de indicar que no tienen ese valor, es decir, es un valor nulo. 
Por lo tanto vamos a sustituirlo por nulos de numpy (np.nan)

In [23]:
bank["pdays"] = bank["pdays"].replace(999, np.nan)
bank.head(3)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,date,latitude,longitude,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,contact_month,contact_year
0,NaN,housemaid,married,basic 4y,No,No,No,telephone,261,1,NaN,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2019-08-02,41.495,-71.233,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,agosto,2019
1,57.0,services,married,high school,NaN,No,No,telephone,149,1,NaN,0,nonexistent,1.1,93.994,-36.4,NaN,5191.0,no,2016-09-14,34.601,-83.923,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,septiembre,2016
2,37.0,services,married,high school,No,Si,No,telephone,226,1,NaN,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2019-02-15,34.939,-94.847,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b,febrero,2019


In [24]:
# Guardar los datos limpios como archivo csv 'bank-limpio.csv'
bank.to_csv('DataSource/bank-limpio.csv')